In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Baseline: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb

# Getting Started with Text Embeddings + Vertex AI Vector Search

#### Install Python SDK

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform google-cloud-storage 'google-cloud-bigquery[pandas]'

## Restart current runtime

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Environment variables

In [ ]:
###### get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [ ]:
###### define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "jc-gcp-project"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

## Enable APIs

In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

## Data Preparation

In [ ]:
###### load the BQ Table into a Pandas DataFrame
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

# Perform a query
QUERY = (
    'select * from `jc-gcp-project.movielens.sample_movies`'
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

df = rows.to_dataframe()

# examine the data
df.head()

id                                              title  popularity  \
0  371085                                          Sharkskin    0.027801   
1  198370                                     Mutual Friends    0.136721   
2   38786  The Blood of My Brother: A Story of Death in Iraq    0.005256   
3  219716                                           Sparkler    0.547654   
4  331493                            Light from the Darkroom    0.012942   

   vote_average                                           overview genre  
0           0.0  The Post War II story of Manhattan born Mike E...  None  
1           0.0  Surprise parties rarely go well. This one is n...  None  
2           0.0  THE BLOOD OF MY BROTHER goes behind the scenes...  None  
3           0.0  Melba is a Californian trailer-park girl who i...  None  
4           0.0  Light in the Darkroom is the story of two best...  None

In [ ]:
len(df)

4803

In [ ]:
df.isna().sum()

id               0
title            0
popularity       0
vote_average     0
overview         3
genre           28
dtype: int64

In [ ]:
# Remove NA
df['overview'] = df['overview'].fillna('')
df['genre'] = df['genre'].fillna('')

In [ ]:
df[df['title'] == 'Avatar']

id   title  popularity  vote_average  \
2228  19995  Avatar  150.437577           7.2   

                                               overview   genre  
2228  In the 22nd century, a paraplegic Marine is di...  Action

## Call the API to generate embeddings

In [ ]:
###### init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
###### Load the text embeddings model
from vertexai.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@001")

## API Usage Quota

In [ ]:
###### Functions

import time

import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs

In [ ]:
df['text'] = '"title" : ' + df['title'].astype("string") + ', "popularity" : ' + df['popularity'].astype("string") + ', "vote_average"' + df['vote_average'].astype("string") + ', "genre" : ' + df['genre'].astype("string") + ', "overview" : ' + df['overview'].astype("string")

In [ ]:
# get embeddings for the question titles and add them as "embedding" column
df = df.assign(embedding=get_embeddings_wrapper(list(df.text)))
df.head()

100%|██████████| 961/961 [17:31<00:00,  1.09s/it]


id                                              title  popularity  \
0  371085                                          Sharkskin    0.027801   
1  198370                                     Mutual Friends    0.136721   
2   38786  The Blood of My Brother: A Story of Death in Iraq    0.005256   
3  219716                                           Sparkler    0.547654   
4  331493                            Light from the Darkroom    0.012942   

   vote_average                                           overview genre  \
0           0.0  The Post War II story of Manhattan born Mike E...         
1           0.0  Surprise parties rarely go well. This one is n...         
2           0.0  THE BLOOD OF MY BROTHER goes behind the scenes...         
3           0.0  Melba is a Californian trailer-park girl who i...         
4           0.0  Light in the Darkroom is the story of two best...         

                                                text  \
0  "title" : Sharkskin, "popularity" : 0.027801, ...   
1  "title" : Mutual Friends, "popularity" : 0.136...   
2  "title" : The Blood of My Brother: A Story of ...   
3  "title" : Sparkler, "popularity" : 0.547654, "...   
4  "title" : Light from the Darkroom, "popularity...   

                                           embedding  
0  [0.015336157754063606, -0.038394711911678314, ...  
1  [0.009683111682534218, -0.04617595672607422, 0...  
2  [-0.029223162680864334, -0.033517830073833466,...  
3  [0.04448612406849861, -0.0565938837826252, 0.0...  
4  [-0.02965383604168892, -0.05053010955452919, 0...

In [ ]:
# BigQuery 테이블 저장!
project_id = 'jc-gcp-project'  # 프로젝트 ID
dataset_id = 'sec_test'  # 데이터셋 ID
table_id = 'movie_embeddings'  # 테이블 ID

# DataFrame을 BigQuery 테이블로 저장
df.to_gbq(f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

<ipython-input-14-d14fd032eae4>:7: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 8112.77it/s]


In [ ]:
# df[df['title'] == 'Avatar'].embedding.iloc[0] # avartar 임베딩
df[df['title'] == 'Avatar']

id   title  popularity  vote_average  \
2228  19995  Avatar  150.437577           7.2   

                                               overview   genre  \
2228  In the 22nd century, a paraplegic Marine is di...  Action   

                                                   text  \
2228  "title" : Avatar, "popularity" : 150.437577, "...   

                                              embedding  
2228  [0.018325019627809525, -0.054211799055337906, ...

In [ ]:
## 아바타 샘플 임베딩 값 확인
# df[df['title'] == 'Avatar'].embedding.iloc[0]

## Get Randome one & Check similarity w/ Dot

In [ ]:
import random
import numpy as np

# pick one of them as a key question
# key = random.randint(0, len(df))
key = df[df['title'] == 'Avatar'].index[0]
# calc dot product between the key and other questions
embs = np.array(df.embedding.to_list())
similarities = np.dot(embs[key], embs.T)

# print similarities for the first 5 questions
similarities[:5]

array([0.60934513, 0.58459621, 0.6016171 , 0.64416181, 0.58156595])

## Print the list

In [ ]:
# print the question
print(f"Key question: {df.title[key]}\n")

# sort and print the questions by similarities
sorted_questions = sorted(
    zip(df.title, similarities), key=lambda x: x[1], reverse=True
)[:20]
for i, (question, similarity) in enumerate(sorted_questions):
    print(f"{similarity:.4f} {question}")

Key question: Avatar

1.0000 Avatar
0.7709 The Last Airbender
0.7398 Titan A.E.
0.7379 The Inhabited Island
0.7375 John Carter
0.7319 Pandorum
0.7306 Planet of the Apes
0.7279 Conquest of the Planet of the Apes
0.7179 Beneath the Planet of the Apes
0.7177 Avengers: Age of Ultron
0.7175 Escape from Planet Earth
0.7166 Oblivion
0.7160 Apocalypto
0.7153 Lost in Space
0.7150 Captain America: Civil War
0.7149 The Martian
0.7148 The Matrix
0.7134 Galaxina
0.7120 Cargo
0.7110 Mission to Mars


## Find with Natural Language

In [ ]:
want_to_find = "외계인 나오는 영화"

emb_val = model.get_embeddings(
    [want_to_find],
)
# calc dot product between the key and other questions
embs = np.array(df.embedding.to_list())
similarities = np.dot(emb_val[0].values, embs.T)
# print the question
print(f"Key question: {want_to_find}\n")

# sort and print the questions by similarities
sorted_questions = sorted(
    zip(df.title, similarities), key=lambda x: x[1], reverse=True
)[:20]
for i, (question, similarity) in enumerate(sorted_questions):
    print(f"{similarity:.4f} {question}")

Key question: 외계인 나오는 영화

0.6784 Monsters
0.6664 Alien
0.6635 Aliens
0.6599 U.F.O.
0.6587 My Stepmother is an Alien
0.6551 Invaders from Mars
0.6551 Spaced Invaders
0.6523 They Live
0.6512 Unknown
0.6511 The Thing
0.6474 Muppets from Space
0.6474 What Planet Are You From?
0.6446 The Outsiders
0.6439 Contact
0.6434 Meteor
0.6423 The Watch
0.6379 Meet Dave
0.6364 Independence Day: Resurgence
0.6346 Surrogates
0.6336 Gods and Monsters


# (Optional) - Vector Search

## Save the embeddings in a JSON file

In [ ]:
# save id and embedding as a json file
jsonl_string = df[["id", "embedding"]].to_json(orient="records", lines=True)
with open("questions.json", "w") as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 questions.json

{"id":371085,"embedding":[0.0153361578,-0.0383947119,0.0352433771,0.0147550041,-0.0319922231,-0.0459211282,-0.0205535181,-0.041891966,0.0089921989,0.0284238458,0.014186698,0.0300955791,0.0258459207,-0.0378358439,0.0237927306,0.0335237756,0.0195060074,0.0082418136,-0.0965101644,0.019350538,-0.0078775575,-0.0119496165,-0.0274844002,0.0447204672,0.0143402014,-0.0388614349,-0.0185875259,-0.0016751911,-0.0199454874,-0.0016352511,0.0079187471,-0.0133166835,0.0349910371,0.0353308655,0.040520031,0.0601883009,0.0099755321,-0.0040698647,0.0197575428,0.0556138679,0.056258142,-0.0870138109,-0.0106943715,0.0019231349,0.0433021486,-0.0126744583,-0.0298540033,0.0327363946,0.0133010373,0.0056282762,0.0609988943,-0.0264207441,0.0384953767,0.0040295478,0.0478877015,0.0103862984,-0.0204253197,0.0240118429,-0.0155596314,0.0039435392,0.0222576689,-0.055673115,0.0395441763,0.0292392205,-0.0050922697,0.032362625,-0.0595636182,0.0457811058,0.005710192,0.0020742365,0.0170880556,0.0225609373,-0.0893099606,0.028

## Save to Bucket

In [ ]:
BUCKET_URI = f"gs://{PROJECT_ID}-movie_embedding-{UID}"
! gsutil mb -l $LOCATION -p {PROJECT_ID} {BUCKET_URI}
! gsutil cp questions.json {BUCKET_URI}

Creating gs://jc-gcp-project-movie_embedding-12010905/...
Copying file://questions.json [Content-Type=application/json]...
- [1 files][ 47.2 MiB/ 47.2 MiB]                                                
Operation completed over 1 objects/47.2 MiB.                                     


## Create an Index

In [ ]:
###### init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"embvs-movie-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=20,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/78565683329/locations/us-central1/indexes/7527375351010295808/operations/2693361931053432832
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/78565683329/locations/us-central1/indexes/7527375351010295808
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/78565683329/locations/us-central1/indexes/7527375351010295808')


### **The parameters for creating index**
#### **contents_delta_uri**: The URI of Cloud Storage directory where you stored the embedding JSON files
#### **dimensions**: Dimension size of each embedding. In this case, it is 768 as we are using the embeddings from the Text Embeddings API.
#### **approximate_neighbors_count**: how many similar items we want to retrieve in typical cases
#### **distance_measure_type**: what metrics to measure distance/similarity between embeddings. In this case it's DOT_PRODUCT_DISTANCE

## Create Index Endpoint and deploy the Index

In [ ]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"movie_indexing_endpoint-{UID}",
    public_endpoint_enabled=True,
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888/operations/6935893517524795392
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888')


In [ ]:
DEPLOYED_INDEX_ID = f"movie_indexing_test_deployed_2"

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888/operations/3585778344714567680
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888


resource name: projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888

## Run Query

In [ ]:
###### Call Endpoint
from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT="1051074875.us-central1-78565683329.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/78565683329/locations/us-central1/indexEndpoints/664311731362725888"
DEPLOYED_INDEX_ID="search_test_1737044762588"

In [ ]:
###### Call Endpoint
PROJECT_ID = "jc-gcp-project"
REGION = "us-central1"
ENDPOINT_ID = "664311731362725888"

In [ ]:
###### Call Endpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name=f"projects/{PROJECT_ID}/locations/{REGION}/indexEndpoints/{ENDPOINT_ID}"
)

In [ ]:
###### Test
query = '니가 가라 하와이'
test_embeddings = get_embeddings_wrapper([query])

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


In [ ]:
###### Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

0.6365 Nim's Island
0.6295 Rapa Nui
0.6244 A Perfect Getaway
0.6213 Treading Water
0.6098 Aloha
0.6077 Yes
0.6030 Beyond the Lights
0.6018 Six Days Seven Nights
0.6006 Just Go with It
0.5988 Dil Jo Bhi Kahey...
0.5979 Princess Kaiulani
0.5975 Neal 'n' Nikki
0.5975 Whale Rider
0.5971 Forgetting Sarah Marshall
0.5945 From Here to Eternity
0.5879 Showgirls
0.5829 Here On Earth
0.5826 Letters from Iwo Jima
0.5822 Flags of Our Fathers
0.5819 The Beach


In [ ]:
# @title Agent initialization.
######

from IPython.display import display, Markdown
from vertexai.preview.generative_models import grounding

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="jc-gcp-project"
REGION="us-central1"

vertexai.init(project=PROJECT_ID, location=REGION)
model_1 = GenerativeModel(MODEL_NAME)

In [ ]:
# @title Call Gemini
######
def give_to_gemini(query):
  prompt =f"""
          당신은 영화정보를 제공하는AI 어시스턴트 입니다. 답변은 질문 내용을 참고해서 답해주세요.
          답변은 3개의 영화를 알려주세요. 질문과 제목이 똑같은 영화는 추천하지 마세요.

          답변은 JSON 포맷으로 아래와 같이 답변해 주세요.

              "title": "movie title",
              "type": "Genre",
              "release_date": "date",
              "director": "director name",
              "cast": "chracters name"

          <질문> : {query}
  """
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model_1.generate_content(
      [prompt],
      tools=[tool],

  )
  return responses.text

In [ ]:
# # @title Call Gemini
# ######
# def give_to_gemini(query):
#   prompt =f"""
#           당신은 영화정보를 제공하는AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

#           1. 영화 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
#           2. 영화를 선정 하는 이유를 설명 해주세요.
#           3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

#           <질문> : {query}
#   """
#   tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

#   responses = model_1.generate_content(
#       [prompt],
#       tools=[tool],

#   )
#   return responses.text

In [ ]:
for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")
    if neighbor.distance < 0.7:
        print("stop and give to Gemini")
        print("")
        ans = give_to_gemini(query)
        print(ans)
        break

0.6365 Nim's Island
stop and give to Gemini

```json
[
  {
    "title": "친구",
    "type": "범죄, 드라마",
    "release_date": "2001-04-14",
    "director": "곽경택",
    "cast": "장동건, 유오성, 김원해, 정진영"
  },
  {
    "title": "아바타",
    "type": "SF, 액션, 모험",
    "release_date": "2009-12-16",
    "director": "제임스 카메론",
    "cast": "샘 워싱턴, 조 샐다나, 시고니 위버, 스티븐 랭"
  },
  {
    "title": "인셉션",
    "type": "SF, 액션, 스릴러",
    "release_date": "2010-07-15",
    "director": "크리스토퍼 놀란",
    "cast": "레오나르도 디카프리오, 조셉 고든-레빗, 엘렌 페이지, 마리옹 꼬티아르"
  }
]
```


## Clean Up

In [ ]:
# # wait for a confirmation
# input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# # delete Index Endpoint
# my_index_endpoint.undeploy_all()
# my_index_endpoint.delete(force=True)

# # delete Index
# my_index.delete()

# # delete Cloud Storage bucket
# ! gsutil rm -r {BUCKET_URI}

Structured search, Vector search

Vertex AI Search (콜수) - 빅쿼리에 데이터 넣고, 인덱싱해서 따로 만듦
임베딩할 필요없음
BQ, CloudSQL